## 1. Install Necessary Libraries

In [ ]:
!pip install boto3 PyMuPDF requests faiss-cpu opensearch-py numpy anthropic

## 2. Setup AWS Bedrock, OpenSearch, and Other APIs

In [ ]:
import boto3
import requests
import fitz  # PyMuPDF
import numpy as np
from opensearchpy import OpenSearch, helpers
from anthropic import Client as AnthropicClient

# Hardcoded AWS Credentials (replace with your credentials)
aws_access_key = 'YOUR_AWS_ACCESS_KEY'
aws_secret_key = 'YOUR_AWS_SECRET_KEY'

# Initialize Boto3 Client for Bedrock (For Amazon Titan embeddings)
bedrock = boto3.client(
    'bedrock',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name='us-east-1'  # Adjust region if needed
)

# Initialize OpenSearch Client
opensearch = OpenSearch(
    hosts=[{'host': 'your-opensearch-endpoint', 'port': 443}],
    http_auth=(aws_access_key, aws_secret_key),
    use_ssl=True,
    verify_certs=True
)

# Initialize Anthropic Client
anthropic_client = AnthropicClient(api_key="YOUR_ANTHROPIC_API_KEY")


## 3. Extract Text, Images, and Hyperlinks from PDF

In [ ]:
def extract_text_and_links_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    links = []
    images = []

    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text("text")
        links += page.get_links()
        
        # Extract images
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = document.extract_image(xref)
            images.append(base_image['image'])
    
    return text, links, images


## 4. Process Hyperlinks and Fetch Content

In [ ]:
from bs4 import BeautifulSoup

def fetch_content_from_link(link):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup.get_text()
        return None
    except Exception as e:
        print(f"Error fetching link {link}: {e}")
        return None


## 5. Use Recursive Character Splitter for Text Segmentation

In [ ]:
def recursive_split(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


## 6. Index Text into OpenSearch Vector Database

In [ ]:
def index_text_to_opensearch(chunks, embedding_model, opensearch_client, index_name="documents"):
    # Ensure index exists
    if not opensearch_client.indices.exists(index=index_name):
        opensearch_client.indices.create(index=index_name, body={"settings": {"index": {"number_of_shards": 1}}})
    
    for chunk in chunks:
        embedding = get_embedding(chunk, embedding_model)
        doc = {
            "text": chunk,
            "embedding": embedding
        }
        opensearch_client.index(index=index_name, body=doc)

def get_embedding(text, embedding_model):
    # Use Amazon Titan or a similar embedding model from Bedrock API
    response = bedrock.invoke_model(model_id="amazon.titan", input={"text": text})
    return response["embedding"]


## 7. Retrieve Relevant Chunks from OpenSearch

In [ ]:
def retrieve_relevant_chunks(query, opensearch_client, embedding_model, index_name="documents"):
    query_embedding = get_embedding(query, embedding_model)
    
    search_body = {
        "query": {
            "knn": {
                "embedding": {
                    "vector": query_embedding,
                    "k": 5  # Retrieve top 5 matches
                }
            }
        }
    }

    response = opensearch_client.search(index=index_name, body=search_body)
    return [hit["_source"]["text"] for hit in response["hits"]["hits"]]


## 8. ReAct Agent Logic (Reason and Act)

In [ ]:
def generate_response(query, relevant_chunks, anthropic_client, chat_history):
    context = "\n".join(relevant_chunks)
    prompt = f"Given the following context: {context}\nAnswer the following question: {query}"
    
    # Include chat history
    full_prompt = "\n".join(chat_history + [prompt])

    response = anthropic_client.completion(
        model="claude-v1",  # Select appropriate Claude model
        prompt=full_prompt,
        max_tokens=300
    )
    
    # Add the response to the chat history
    chat_history.append(f"User: {query}")
    chat_history.append(f"Claude: {response['completion']}")
    
    return response["completion"]


## 9. Memory Buffer for Chat History

In [ ]:
chat_history = []

def add_to_chat_history(query, response):
    chat_history.append({"query": query, "response": response})


## 10. Evaluation Framework (Grounding and Hallucination Detection)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_grounding(response, relevant_chunks, embedding_model):
    response_embedding = get_embedding(response, embedding_model)
    chunk_embeddings = [get_embedding(chunk, embedding_model) for chunk in relevant_chunks]
    
    similarities = [cosine_similarity([response_embedding], [chunk_embedding])[0][0] for chunk_embedding in chunk_embeddings]
    return np.mean(similarities)

def check_hallucination(response, relevant_chunks):
    for chunk in relevant_chunks:
        if response in chunk:
            return False
    return True

def final_check(response, relevant_chunks, embedding_model):
    grounding_score = evaluate_grounding(response, relevant_chunks, embedding_model)
    hallucination_detected = check_hallucination(response, relevant_chunks)
    
    if hallucination_detected or grounding_score < 0.5:
        return "Hmm, I don't know"
    return response


## 11. Main Application Logic

In [ ]:
def main(pdf_path, query):
    # Step 1: Extract text, images, and hyperlinks from the PDF
    text, links, images = extract_text_and_links_from_pdf(pdf_path)
    
    # Step 2: Split text using recursive splitter
    text_chunks = recursive_split(text)
    
    # Step 3: Fetch content from hyperlinks and add to chunks
    for link in links:
        if 'uri' in link:
            content = fetch_content_from_link(link['uri'])
            if content:
                link_chunks = recursive_split(content)
                text_chunks.extend(link_chunks)
    
    # Step 4: Index chunks into OpenSearch
    index_text_to_opensearch(text_chunks, get_embedding, opensearch, index_name="documents")
    
    # Step 5: Retrieve relevant chunks
    relevant_chunks = retrieve_relevant_chunks(query, opensearch, get_embedding, index_name="documents")
    
    # Step 6: Generate response using ReAct agent and Anthropic
    response = generate_response(query, relevant_chunks, anthropic_client, chat_history)
    
    # Step 7: Evaluate and detect hallucination
    final_response = final_check(response, relevant_chunks, get_embedding)
    
    # Step 8: Display response
    print(final_response)

# Execute
if __name__ == "__main__":
    pdf_path = "path_to_your_pdf_file.pdf"
    query = "Your question here"
    main(pdf_path, query)
